In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Data Reading**

In [ ]:
mata_data1_path = "../input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv"
mata_data2_path = "../input/learnplatform-covid19-impact-on-digital-learning/products_info.csv"


In [ ]:
meta_data1 = pd.read_csv(mata_data1_path)
meta_data2 = pd.read_csv(mata_data2_path)


**Processing Engegement Data**

**Concatinating Engagement Data**

columns in Engagement Dataframe
* time
* lp_id
* pct_access
* engagement_index
* district_id

In [ ]:
engagement_Path = '../input/learnplatform-covid19-impact-on-digital-learning/engagement_data' 

temp = []

for district in meta_data1.district_id.unique():
    df = pd.read_csv(f'{engagement_Path}/{district}.csv', index_col=None, header=0)
    df["district_id"] = district
    temp.append(df)
    
    
engagement_df = pd.concat(temp)
engagement_df = engagement_df.reset_index(drop=True)
engagement_df

**Missing Values**

In [ ]:
engagement_df.isna()

**Missing Values With in the column**

In [ ]:
missing_value=engagement_df.isna().sum()
missing_value

In [ ]:
# Function to calculate missing values by column
def missing_values_table(df):
    # Total missing values
    mis_val = engagement_df.isnull().sum()

    # Percentage of missing values
    mis_val_percent = 100 * engagement_df.isnull().sum() / len(df)

    # dtype of missing values
    mis_val_dtype = engagement_df.dtypes

    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent, mis_val_dtype], axis=1)

    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values', 2: 'Dtype'})

    # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)

    # Print some summary information
    print ("Your selected dataframe has " + str(engagement_df.shape[1]) + " columns.\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) + " columns that have missing values.")

    # Return the dataframe with missing information
    return mis_val_table_ren_columns

print(missing_values_table(engagement_df))

**Fill Missing Values**

In [ ]:
import pandas as pd

class Clean_data:
    
    def __init__(self, df:pd.DataFrame):
        self.df = df
        
    def Fiil_value(self, df: pd.DataFrame) -> pd.DataFrame:

        try:
            #df.dropna()
            df.fillna(0,inplace = True)
            #df.replace(to_replace = np.nan, value = 0 )
        except:
            pass
        
        return df

    def drop_value(self, df:pd.DataFrame)->pd.DataFrame:


        try:
            df = df[df.state.notna()].reset_index(drop=True)
            for column in df.columns:
                df[column].fillna(df[column].mode()[0], inplace=True)
            #df.dropna()
        except:
            pass

        return df
    def duplicates(self, df:pd.DataFrame)->pd.DataFrame:
        try:
            df.drop_duplicates(inplace=True)
            
        except:
            pass
        
        return df

**filling missing values with 0**

In [ ]:
clean_meta_data1= Clean_data(engagement_df)
df = clean_meta_data1.Fiil_value(engagement_df)
df.head()

**Processing Districts Data**

In [ ]:
mata_data1_path = "../input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv"

In [ ]:
district_data = pd.read_csv(mata_data1_path)
district_data

the District data contains the folowing columns 
* district_id
* state	locale
* pct_black/hispanic
* pct_free/reduced
* county_connections_ratio
* pp_total_raw

**Missing value treatment**

filing missing values with mode of the cloumn

In [ ]:
clean_meta_data1= Clean_data(district_data)
df = clean_meta_data1.drop_value(district_data)
df

**Processing Product Data**

In [ ]:
mata_data2_path = "../input/learnplatform-covid19-impact-on-digital-learning/products_info.csv"

In [ ]:
product_df = pd.read_csv(mata_data2_path)
#rename column to merge
product_data1 = product_df.rename({'LP ID':'lp_id'}, axis=1)
# product_df
product_data1

In [ ]:
print(missing_values_table(product_data1))

we have 0 missing value in product_info dataframe

**Joining District_info and engagement datasets**

In [ ]:
eng_dist_df = pd.merge(engagement_df,district_data, on = "district_id" )
eng_dist_pro_df = pd.merge(eng_dist_df,product_data1, on = "lp_id" )
eng_dist_pro_df 